In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM,Dense,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.models import Sequential 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data=pd.read_csv("/kaggle/input/capital-dataset/Capital_Project_Detail_Data_-_Dollars_20250125.csv")

In [3]:
# Set all non-NaN values in DELAY_DESC to 1
data["DELAY_DESC"] = data["DELAY_DESC"].apply(lambda x: "delay" if pd.notnull(x) else x)

In [4]:
data["DELAY_DESC"].value_counts()

DELAY_DESC
delay    43719
Name: count, dtype: int64

In [5]:
data["DELAY_DESC"].isnull().sum()
data["DELAY_DESC"].fillna("No Delay", inplace=True)

<ipython-input-5-5105570d60f9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["DELAY_DESC"].fillna("No Delay", inplace=True)


In [6]:
data.head(10)

,PUB_DATE,BORO,MANAGING_AGCY_CD,MANAGING_AGCY,PROJECT_ID,PROJECT_DESCR,TYP_CATEGORY_NAME,COMMUNITY_BOARD,BUDGET_LINE,DELAY_DESC,...,CITY_RTC,CITY_PLAN_TOTAL,NONCITY_PRIOR_ACTUAL,NONCITY_YR1_PLAN,NONCITY_YR2_PLAN,NONCITY_YR3_PLAN,NONCITY_YR4_PLAN,NONCITY_YR5_PLAN,NONCITY_RTC,NONCITY_PLAN_TOTAL
0,20200116,CITYWIDE,42,CITY UNIVERSITY,CA207-006,CUNY Incubator Network,MISCELLANEOUS RECONSTRUCTION,0,HN-0206,delay,...,0,7160.0,0.0,0,0,0,0,0,0,0.0
1,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79-ADA,ADA COMPLIANCE RENOVATIONS AT NYPD PRECINCTS,POLICE FACILITIES,0,PO-0079,No Delay,...,0,29337.0,0.0,0,0,0,0,0,0,0.0
2,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BATH,CITYWIDE BATHROOM RENOVATION,POLICE FACILITIES,0,PO-0079,delay,...,0,37118.0,0.0,0,0,0,0,0,0,0.0
3,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAHD,"BATHROOM RENO AT 67TH, 77TH AND TRAFFIC CONTRO...",POLICE FACILITIES,0,PO-0079,delay,...,0,1407.0,0.0,0,0,0,0,0,0,0.0
4,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAHG,"ELEVATOR REHABILITATION FOR 20TH, 25TH",POLICE FACILITIES,0,PO-0079,delay,...,0,2500.0,0.0,0,0,0,0,0,0,0.0
5,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAHI,"BOILER REPLACEMENT AT 68TH, 76TH AND 102ND PCTS",POLICE FACILITIES,0,PO-0079,delay,...,0,2430.0,0.0,0,0,0,0,0,0,0.0
6,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAHY,"BOILERS AT 26TH, 42ND & 46TH PRECINCTS",POLICE FACILITIES,0,PO-0079,delay,...,0,1258.0,0.0,0,0,0,0,0,0,0.0
7,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAJZ,49TH & 115TH & MTN PCT HVAC REPLACEMENT,POLICE FACILITIES,0,PO-0079,No Delay,...,0,2372.0,0.0,0,0,0,0,0,0,0.0
8,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAKF,CAC BATHROOM RENOVATION,POLICE FACILITIES,0,PO-0079,No Delay,...,0,487.0,0.0,0,0,0,0,0,0,0.0
9,20200116,CITYWIDE,56,POLICE DEPARTMENT,PO79BMAKI,"BOILERS AT MTS, HWY 3, 75 & 77 PRECINCTS",POLICE FACILITIES,0,PO-0079,No Delay,...,0,3268.0,0.0,0,0,0,0,0,0,0.0


In [7]:
data.columns

Index(['PUB_DATE', 'BORO', 'MANAGING_AGCY_CD', 'MANAGING_AGCY', 'PROJECT_ID',
       'PROJECT_DESCR', 'TYP_CATEGORY_NAME', 'COMMUNITY_BOARD', 'BUDGET_LINE',
       'DELAY_DESC', 'SITE_DESCR', 'SCOPE_TEXT', 'FY_YR1_PLAN', 'ORIG_BUD_AMT',
       'CITY_PRIOR_ACTUAL', 'CITY_YR1_PLAN', 'CITY_YR2_PLAN', 'CITY_YR3_PLAN',
       'CITY_YR4_PLAN', 'CITY_YR5_PLAN', 'CITY_RTC', 'CITY_PLAN_TOTAL',
       'NONCITY_PRIOR_ACTUAL', 'NONCITY_YR1_PLAN', 'NONCITY_YR2_PLAN',
       'NONCITY_YR3_PLAN', 'NONCITY_YR4_PLAN', 'NONCITY_YR5_PLAN',
       'NONCITY_RTC', 'NONCITY_PLAN_TOTAL'],
      dtype='object')

In [8]:
data=data.drop("SCOPE_TEXT",axis=1)

In [9]:
data = data.iloc[:, :-8]
data.isnull().sum()

PUB_DATE                 0
BORO                     0
MANAGING_AGCY_CD         0
MANAGING_AGCY            0
PROJECT_ID               0
PROJECT_DESCR            0
TYP_CATEGORY_NAME        0
COMMUNITY_BOARD       6510
BUDGET_LINE              0
DELAY_DESC               0
SITE_DESCR           48221
FY_YR1_PLAN              0
ORIG_BUD_AMT             0
CITY_PRIOR_ACTUAL        0
CITY_YR1_PLAN            0
CITY_YR2_PLAN            0
CITY_YR3_PLAN            0
CITY_YR4_PLAN            0
CITY_YR5_PLAN            0
CITY_RTC                 0
CITY_PLAN_TOTAL          0
dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()


columns_to_encode = ["BORO", "MANAGING_AGCY", "PROJECT_DESCR","PROJECT_ID","COMMUNITY_BOARD",
                     "TYP_CATEGORY_NAME", "BUDGET_LINE", "DELAY_DESC", "SITE_DESCR"]

for col in columns_to_encode:
    data[col] = encoder.fit_transform(data[col])


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72437 entries, 0 to 72436
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PUB_DATE           72437 non-null  int64  
 1   BORO               72437 non-null  int64  
 2   MANAGING_AGCY_CD   72437 non-null  int64  
 3   MANAGING_AGCY      72437 non-null  int64  
 4   PROJECT_ID         72437 non-null  int64  
 5   PROJECT_DESCR      72437 non-null  int64  
 6   TYP_CATEGORY_NAME  72437 non-null  int64  
 7   COMMUNITY_BOARD    72437 non-null  int64  
 8   BUDGET_LINE        72437 non-null  int64  
 9   DELAY_DESC         72437 non-null  int64  
 10  SITE_DESCR         72437 non-null  int64  
 11  FY_YR1_PLAN        72437 non-null  int64  
 12  ORIG_BUD_AMT       72437 non-null  int64  
 13  CITY_PRIOR_ACTUAL  72437 non-null  float64
 14  CITY_YR1_PLAN      72437 non-null  int64  
 15  CITY_YR2_PLAN      72437 non-null  int64  
 16  CITY_YR3_PLAN      724

In [12]:
data.isnull().sum()

PUB_DATE             0
BORO                 0
MANAGING_AGCY_CD     0
MANAGING_AGCY        0
PROJECT_ID           0
PROJECT_DESCR        0
TYP_CATEGORY_NAME    0
COMMUNITY_BOARD      0
BUDGET_LINE          0
DELAY_DESC           0
SITE_DESCR           0
FY_YR1_PLAN          0
ORIG_BUD_AMT         0
CITY_PRIOR_ACTUAL    0
CITY_YR1_PLAN        0
CITY_YR2_PLAN        0
CITY_YR3_PLAN        0
CITY_YR4_PLAN        0
CITY_YR5_PLAN        0
CITY_RTC             0
CITY_PLAN_TOTAL      0
dtype: int64

In [13]:
data["DELAY_DESC"].value_counts().sum()

72437

In [14]:
data["PUB_DATE"]=pd.to_datetime(data["PUB_DATE"])

In [15]:
import pandas as pd

data["PUB_DATE"] = pd.to_datetime(data["PUB_DATE"], format="%Y%m%d")

In [16]:
data["month"]=data["PUB_DATE"].dt.month
data["day"]=data["PUB_DATE"].dt.day
data["year"]=data["PUB_DATE"].dt.year
data=data.drop("PUB_DATE",axis=1)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72437 entries, 0 to 72436
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BORO               72437 non-null  int64  
 1   MANAGING_AGCY_CD   72437 non-null  int64  
 2   MANAGING_AGCY      72437 non-null  int64  
 3   PROJECT_ID         72437 non-null  int64  
 4   PROJECT_DESCR      72437 non-null  int64  
 5   TYP_CATEGORY_NAME  72437 non-null  int64  
 6   COMMUNITY_BOARD    72437 non-null  int64  
 7   BUDGET_LINE        72437 non-null  int64  
 8   DELAY_DESC         72437 non-null  int64  
 9   SITE_DESCR         72437 non-null  int64  
 10  FY_YR1_PLAN        72437 non-null  int64  
 11  ORIG_BUD_AMT       72437 non-null  int64  
 12  CITY_PRIOR_ACTUAL  72437 non-null  float64
 13  CITY_YR1_PLAN      72437 non-null  int64  
 14  CITY_YR2_PLAN      72437 non-null  int64  
 15  CITY_YR3_PLAN      72437 non-null  int64  
 16  CITY_YR4_PLAN      724

In [18]:
linear_ydata=data["CITY_PLAN_TOTAL"]
linear_xdata=data.drop("CITY_PLAN_TOTAL",axis=1)

In [19]:
x_train,x_test,y_train,y_test=train_test_split(linear_xdata,linear_ydata,test_size=0.2)

In [20]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(57949, 22)
(57949,)
(14488, 22)
(14488,)


In [21]:
import numpy as np

X_train = np.array(x_train).reshape((x_train.shape[0], 1, x_train.shape[1]))
X_test = np.array(x_test).reshape((x_test.shape[0], 1, x_test.shape[1]))

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),  
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')  # Assuming a binary classification problem
])


model.compile(optimizer='adam', loss='mae', metrics=['mae'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 10882.9795 - mae: 10882.9795 - val_loss: 3113.3562 - val_mae: 3113.3562
Epoch 2/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 5282.8687 - mae: 5282.8687 - val_loss: 3867.6885 - val_mae: 3867.6885
Epoch 3/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 5102.6685 - mae: 5102.6685 - val_loss: 2232.5745 - val_mae: 2232.5745
Epoch 4/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4007.6965 - mae: 4007.6965 - val_loss: 1986.7802 - val_mae: 1986.7802
Epoch 5/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4176.1694 - mae: 4176.1694 - val_loss: 2413.5359 - val_mae: 2413.5359
Epoch 6/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3824.9202 - mae: 3824.9202 - val_loss: 2594.5166 - val_mae: 2594.5166
Epoch 7/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3919.7190 - mae: 3919.7190 - val_loss: 3038.7412 - val_mae: 3038.7412
Epoch 8/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3754.0474 -

In [24]:
model.save("linear.h5")

In [25]:
cat_ydata=data["DELAY_DESC"]
cat_xdata=data.drop("DELAY_DESC",axis=1)

In [26]:
x_train,x_test,y_train,y_test=train_test_split(linear_xdata,linear_ydata,test_size=0.2)

In [27]:
import numpy as np

X_train = np.array(x_train).reshape((x_train.shape[0], 1, x_train.shape[1]))
X_test = np.array(x_test).reshape((x_test.shape[0], 1, x_test.shape[1]))

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),  # Prevent overfitting
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Assuming a binary classification problem
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 2.7818e-04 - loss: -950257975296.0000 - val_accuracy: 0.0000e+00 - val_loss: -11796777795584.0000
Epoch 2/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 2.1772e-05 - loss: -16075900059648.0000 - val_accuracy: 0.0000e+00 - val_loss: -57941424078848.0000
Epoch 3/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 8.7337e-05 - loss: -62575412772864.0000 - val_accuracy: 0.0000e+00 - val_loss: -148257959313408.0000
Epoch 4/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 6.9036e-05 - loss: -150606434009088.0000 - val_accuracy: 0.0000e+00 - val_loss: -299165108142080.0000
Epoch 5/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 8.3097e-05 - loss: -330268992864256.0000 - val_accuracy: 0.0000e+00 - val_loss: -486767098593280.0000
Epoch 6/20
1811/1811 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.1754e-04 - loss: -521459260719104.0000 - val_accuracy: 0.0000e+00 - val_loss: -746794954784768.000

In [30]:
model.save("cat.h5")

In [31]:
# Define the columns to drop
columns_to_drop = ["CITY_YR1_PLAN", "CITY_YR2_PLAN", "CITY_YR3_PLAN", "CITY_YR4_PLAN", "CITY_YR5_PLAN"]

# Extract target values (Y data)
predict_ydata = data[columns_to_drop]  # Use double brackets to select multiple columns

# Extract feature values (X data)
predict_xdata = data.drop(columns=columns_to_drop, axis=1)

# Check the shapes of the datasets
print("Shape of X data:", predict_xdata.shape)
print("Shape of Y data:", predict_ydata.shape)


Shape of X data: (72437, 18)
Shape of Y data: (72437, 5)


In [32]:
predict_xdata = np.array(predict_xdata).reshape((predict_xdata.shape[0], 1, predict_xdata.shape[1]))
predict_ydata = np.array(predict_ydata)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(predict_xdata, predict_ydata, test_size=0.2, random_state=42)

In [34]:
# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_scaled.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(5, activation='linear')  # 5 outputs for budget predictions
])


NameError: name 'X_scaled' is not defined

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:
model.save('budget_forecasting_lstm.h5')
print("Model saved successfully.")